In [1]:
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from datasets import load_dataset, load_metric
from utils import get_preprocess_function_train, get_preprocess_function_val_test
from t5_lit_mod import LitT5
from torch.utils.data import DataLoader

In [2]:
model_checkpoint = "t5-small"

In [3]:
# Load dataset
raw_datasets = load_dataset("esnli")

Found cached dataset esnli (/home/jeroen-ov/.cache/huggingface/datasets/esnli/plain_text/0.0.2/a160e6a02bbb8d828c738918dafec4e7d298782c334b5109af632fec6d779bbc)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer: T5Tokenizer = T5Tokenizer.from_pretrained(model_checkpoint, model_max_length=512)
model = LitT5(model_name=model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True, label_pad_token_id=-100)

In [5]:
train_preprocess_function = get_preprocess_function_train(tokenizer)
preprocessed_train_dataset = raw_datasets['train'].map(train_preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

  0%|          | 0/550 [00:00<?, ?ba/s]

In [ ]:
train_dataloader = DataLoader(preprocessed_train_dataset, shuffle=True, batch_size=16, collate_fn=data_collator)

In [ ]:
val_preprocess_function = get_preprocess_function_val_test(tokenizer)
preprocessed_val_dataset = raw_datasets['validation'].map(train_preprocess_function, batched=True, remove_columns=raw_datasets["validation"].column_names)

In [ ]:
# Preprocess dataset
# @TODO write preprocessing function in utils.py
# @NOTE labels should end with eos_token
# Prefix should be added

preprocess_fn = get_preprocess_function(tokenizer)
tokenized_datasets = raw_datasets.map(preprocess_fn, batched=True, remove_columns=raw_datasets["train"].column_names)

In [ ]:
# Load model
# AutoModelForSeq2SeqLM can also be used, but went for T5 specific function. Don't know exact difference.
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

In [ ]:
model_name = model_checkpoint.split("/")[-1]

# Define all the training arguments (these are just taken from tutorial)
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
# Data collector to pad the batches and labels to max length
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our `Seq2SeqTrainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```